# Plot entire city neighborhood by neighborhood

_Tamas Budavari_ <br>
<budavari@jhu.edu> <br>
2018-08-30


In [ ]:
import warnings
warnings.filterwarnings('ignore')

%pylab inline
from SciServer import CasJobs
import pandas as pd
from shapely import wkt

import geopandas as gpd

In [ ]:
# neighborhoods
df = CasJobs.executeQuery("select distinct Neighborhood from RealPropertiesAll", 'City')

In [ ]:
# helper function to get geo data
def GeoQuery(qry, geocolumn='geometry', ctxt='City', crs=2248):
    df = CasJobs.executeQuery(qry, ctxt)
    crsinit = {'init': 'epsg:4326'}
    gdf = gpd.GeoDataFrame(df, crs=crsinit, geometry=df[geocolumn].apply(wkt.loads))
    return gdf.to_crs(epsg=crs)

In [ ]:
#  query template for neighborhood shape
qnb = """
    select geometry=Geog.ToString()
    from Neighborhoods 
"""

# query template for number of vacants, etc., in subblockfaces
qry = """
    select s.MinID, N, Nvac, Nmcc, Nvmc, Nown, Fvmc=convert(float,Nvmc)/N
    , MinBlockLot, MaxBlockLot
    , IsSameStreet=coalesce(IsSameStreet,1)
    , geometry=BldgGeog.ToString()
    from SubBlockFaces s
        cross apply (
            select MinID, MinBlockLot=MIN(BlockLot), MaxBlockLot=MAX(BlockLot)
            from RealGroups g
                join RealPropertiesAll r on r.ID=g.ID
            where s.MinID=g.MinID and Neighborhood like '%s'
            group by g.MinID
        ) x
    where Nvac > 0
"""

In [ ]:
cmap = 'cubehelix_r' # color map
vmin, vmax = 0, 1  # color limits

fig, ax = plt.subplots(1, figsize=(16,12))
ax.set_aspect('equal')

# plot neighborhood outlines (union of parcels)
nbr = GeoQuery(qnb)
nbr.plot(ax=ax, linewidth=0.05, edgecolor='none', color='seashell')

# plot contiguous houses with vacancy
for i,n in enumerate(df.Neighborhood[:]):
    print ('Loading ' , i , '/', df.Neighborhood.size, ' : ', n)
    sbf = GeoQuery(qry % n.replace("'", "''"))
    print ('>>>', sbf.shape[0])
    sbf.plot(ax=ax, linewidth=0.05, edgecolor='black', column='Fvmc', cmap=cmap, vmin=vmin, vmax=vmax)
    
#ax.axis('off')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)  
plt.tight_layout()
#plt.savefig('_city.pdf')
plt.savefig('city.svg', format='svg')
print('Done')